# 地理信息系统设计与开发（属性、SQL查询和创建缓冲区）

In [1]:
import os
import ogr
import osr
import json
import requests
import matplotlib.pyplot as plt
%matplotlib inline

## 定义输入输出和缓冲区距离（由空间参考决定单位）

In [2]:
inputfn = 'test3.shp'
outputBufferfn = 'testBuffer3.shp'
bufferDist = 5

## 读入数据

In [3]:
inputds = ogr.Open(inputfn)
inputlyr = inputds.GetLayer()

## 执行SQL查询（在datasource层面）

In [7]:
result = inputds.ExecuteSQL("select * from test3 where magnitude >= 6")
print(result)

<osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x000001E6C418D0C0> >


## 显示查询结果

In [8]:
resultFeat = result.GetNextFeature()
while resultFeat :
   print(resultFeat.GetField('place')) 
   resultFeat = result.GetNextFeature()

186km E of Taron, Papua New Guinea
205km NW of Saumlaki, Indonesia
89km S of Ierapetra, Greece
132km W of Panguna, Papua New Guinea
25km SE of Ofunato, Japan
206km W of Chichi-shima, Japan
55km NNE of Savannah Bight, Honduras
194km SE of Raoul Island, New Zealand


## 执行属性查询（在layer层面）

In [18]:
inputlyr.SetAttributeFilter("magnitude >= 6")
result=inputlyr

## 显示查询结果

In [19]:
resultFeat = result.GetNextFeature()
while resultFeat :
   print(resultFeat.GetField('place')) 
   resultFeat = result.GetNextFeature()

186km E of Taron, Papua New Guinea
205km NW of Saumlaki, Indonesia
89km S of Ierapetra, Greece
132km W of Panguna, Papua New Guinea
25km SE of Ofunato, Japan
206km W of Chichi-shima, Japan
55km NNE of Savannah Bight, Honduras
194km SE of Raoul Island, New Zealand


## 创建输出文件

In [20]:
shpdriver = ogr.GetDriverByName('ESRI Shapefile')
if os.path.exists(outputBufferfn):
    shpdriver.DeleteDataSource(outputBufferfn)
outputBufferds = shpdriver.CreateDataSource(outputBufferfn)
targetSR = osr.SpatialReference()
targetSR.ImportFromEPSG(4326) #Geo WGS84
bufferlyr = outputBufferds.CreateLayer(outputBufferfn,targetSR,geom_type=ogr.wkbPolygon)
featureDefn = bufferlyr.GetLayerDefn()

## 对查询结果建立缓冲区并输出到文件

In [21]:
for feature in inputlyr:
    ingeom = feature.GetGeometryRef()
    geomBuffer = ingeom.Buffer(bufferDist)

    outFeature = ogr.Feature(featureDefn)
    outFeature.SetGeometry(geomBuffer)
    bufferlyr.CreateFeature(outFeature)
    outFeature = None

In [22]:
outputBufferds=None